This notebook fit multilayer neural network and dump results (as second level feature) to file for later ensembling. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.regularizers import l2, l1
from keras.optimizers import RMSprop, Adam
from tensorflow import set_random_seed
set_random_seed(23333)
np.random.seed(233333)

import time
import sys
import gc
import pickle
sys.version_info

Using TensorFlow backend.


sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)

## Read data from file, select features.

In [2]:
data = pd.read_pickle('../output/data.pkl')
# do not use ID features
data = data[[
    'date_block_num',
    #'shop_id',
    #'item_id',
    'item_cnt_month',
    #'city_code',
    #'item_category_id',
    #'type_code','subtype_code',
    'item_cnt_month_lag_1','item_cnt_month_lag_2','item_cnt_month_lag_3','item_cnt_month_lag_6','item_cnt_month_lag_12',
    'item_avg_sale_last_6', 'item_std_sale_last_6',
    'item_avg_sale_last_12', 'item_std_sale_last_12',
    'shop_avg_sale_last_6', 'shop_std_sale_last_6',
    'shop_avg_sale_last_12', 'shop_std_sale_last_12',
    'category_avg_sale_last_12', 'category_std_sale_last_12',
    'city_avg_sale_last_12', 'city_std_sale_last_12',
    'type_avg_sale_last_12', 'type_std_sale_last_12',
    'subtype_avg_sale_last_12', 'subtype_std_sale_last_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_2','date_item_avg_item_cnt_lag_3','date_item_avg_item_cnt_lag_6','date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1','date_shop_avg_item_cnt_lag_2','date_shop_avg_item_cnt_lag_3','date_shop_avg_item_cnt_lag_6','date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month','year',
    'item_shop_last_sale','item_last_sale',
    'item_shop_first_sale','item_first_sale',
]]

## Set up validation strategy

Validation strategy is 34 month for the test set, 33 month for the validation set and 13-32 months for the train.

In [3]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

del data
gc.collect();

## Neural Network

In [4]:
# define model
def Sales_prediction_model(input_shape):
    in_layer = Input(input_shape)
    x = Dense(16,kernel_initializer='RandomUniform', kernel_regularizer=l2(0.02), activation = "relu")(in_layer)
    x = Dense(8, kernel_initializer='RandomUniform', kernel_regularizer=l2(0.02), activation = "relu")(x)
    x = Dense(1, kernel_initializer='RandomUniform', kernel_regularizer=l2(0.02), activation = "relu")(x)
    
    model = Model(inputs = in_layer, outputs = x, name='Sales_prediction_model')
    return model

# NN cannot take missing values, fill NaN with 0.
X_train.fillna(0,inplace=True)
X_valid.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)

# We do no feature scaling here. 
# Some features like 'item_avg_sale_last_6' are already scaled in feature engineering part.

input_shape = [X_train.shape[1]]
model = Sales_prediction_model(input_shape)
model.compile(optimizer = Adam(lr=0.0005) , loss = ["mse"], metrics=['mse'])
model.fit(X_train, Y_train, validation_data = (X_valid, Y_valid), batch_size = 10000, epochs=5)

Train on 6186922 samples, validate on 238172 samples
Epoch 1/5
6186922/6186922 [==============================] - 25s 4us/step - loss: 0.9759 - mean_squared_error: 0.9336 - val_loss: 0.9587 - val_mean_squared_error: 0.9115
Epoch 2/5
6186922/6186922 [==============================] - 24s 4us/step - loss: 0.8561 - mean_squared_error: 0.8114 - val_loss: 0.9578 - val_mean_squared_error: 0.9142
Epoch 3/5
6186922/6186922 [==============================] - 24s 4us/step - loss: 0.8494 - mean_squared_error: 0.8062 - val_loss: 0.9415 - val_mean_squared_error: 0.8988
Epoch 4/5
6186922/6186922 [==============================] - 24s 4us/step - loss: 0.8467 - mean_squared_error: 0.8035 - val_loss: 0.9388 - val_mean_squared_error: 0.8950
Epoch 5/5
6186922/6186922 [==============================] - 25s 4us/step - loss: 0.8455 - mean_squared_error: 0.8016 - val_loss: 0.9423 - val_mean_squared_error: 0.8979


In [5]:
Y_pred = model.predict(X_valid).clip(0, 20)[:,0]
Y_test = model.predict(X_test).clip(0, 20)[:,0]

X_train_level2 = pd.DataFrame({
    "ID": np.arange(Y_pred.shape[0]), 
    "item_cnt_month": Y_pred
})
X_train_level2.to_csv('../output/nn_valid.csv', index=False)

submission = pd.DataFrame({
    "ID": np.arange(Y_test.shape[0]), 
    "item_cnt_month": Y_test
})
submission.to_csv('../output/nn_submission.csv', index=False)

public LB score: 0.99123